# Dashboard FV – Curvas y KPIs

In [ ]:

import pandas as pd
import plotly.express as px
from ipywidgets import Dropdown, VBox, HBox, Output, HTML, Layout

DATA_PATHS = {
    "Calama": "/home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/calama_pv_results.csv",
    "Salvador": "/home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/salvador_pv_results.csv",
    "Vallenar": "/home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/Vallenar_pv_results.csv"
}
POTENCIA_AC_KW = 41670
LCOE = {"Calama": 0.0574, "Salvador": 0.0576, "Vallenar": 0.0706}

dfs = {k: pd.read_csv(v, parse_dates=['Fecha']) for k, v in DATA_PATHS.items()}
for df in dfs.values():
    df['Año'] = df['Fecha'].dt.year
    df['Potencia_AC_kW'] = df['Potencia_AC_kW'] / 1000

site_dd = Dropdown(options=list(dfs.keys()), description='Sitio:')
year_dd = Dropdown(options=sorted({y for d in dfs.values() for y in d['Año'].unique()}), description='Año:')
out_plot, out_kpi = Output(), Output()

def update(*_):
    site, year = site_dd.value, year_dd.value
    d = dfs[site][dfs[site]['Año'] == year]
    if d.empty:
        with out_plot: out_plot.clear_output(); print('Sin datos')
        return
    daily = d.set_index('Fecha').resample('D').sum()
    daily['CF'] = daily['Potencia_AC_kW'] / (24 * POTENCIA_AC_KW)
    with out_plot:
        out_plot.clear_output()
        fig = px.line(d, x='Fecha', y='Potencia_AC_kW', title=f'Potencia AC Horaria – {site} {year}',
                      labels={'Potencia_AC_kW':'Potencia (kW)'})
        fig.update_layout(height=400)
        fig.show()
    with out_kpi:
        out_kpi.clear_output()
        energia = daily['Potencia_AC_kW'].sum() / 1000
        html = (f"<h3>KPIs – {site} {year}</h3>"
                f"<ul>"
                f"<li><b>Energía anual:</b> {energia:,.1f} MWh</li>"
                f"<li><b>LCOE:</b> {LCOE[site]:.4f} USD/kWh</li>"
                f"<li><b>CF medio:</b> {daily['CF'].mean()*100:.1f}%</li>"
                f"</ul>")
        display(HTML(html))

update()
site_dd.observe(update, names='value')
year_dd.observe(update, names='value')
VBox([HBox([site_dd, year_dd], layout=Layout(justify_content='flex-start')), out_plot, out_kpi])
